In [1]:
# selenium and chromium must be pre-installed
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import traceback
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from fake_useragent import UserAgent
from time import sleep

# Preprocess defenitions

In [2]:
defs = pd.read_csv("all_defs.csv")

In [3]:
words1 = []
words2 = []

for i in range(defs.shape[0]):
    if str(defs["text2"][i]).find("1)") != -1:
        words2.append(defs["word"][i])
    if str(defs["text1"][i]).find("1)") != -1:
        words1.append(defs["word"][i])
        
print(len(words1) / len(defs), len(words2))

0.08793188144751923 100


In [4]:
defs["meanings1"] = None
defs["meanings2"] = None
defs["common_part"] = None

for i in range(defs.shape[0]):
    if str(defs["text1"][i]).find("1)") != -1:
        
        ind = str(defs["text1"][i]).find("1)")
        if ind - 1 > 0:
            common = str(defs["text1"][i])[:ind - 1]    
            defs["common_part"][i] = common
            
        non = str(defs["text1"][i])[ind + 2:]
        for r in range(2, 10):
            non = non.replace(str(r) + ")", "**")
        
        non = non.split("**")
        defs["meanings1"][i] = non
        
    if str(defs["text2"][i]).find("1)") != -1:
        
        ind = str(defs["text2"][i]).find("1)")
        if ind - 1 > 0:
            common = str(defs["text2"][i])[:ind - 1]
        
            defs["common_part"][i] = common
        non = str(defs["text2"][i])[ind + 2:]
        for r in range(2, 10):
            non = non.replace(str(r) + ")", "**")
        
        non = non.split("**")
        defs["meanings2"][i] = non

defs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6107 entries, 0 to 6106
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   6107 non-null   int64 
 1   letter       6107 non-null   object
 2   url          6107 non-null   object
 3   word         6107 non-null   object
 4   text1        6106 non-null   object
 5   text2        1095 non-null   object
 6   meanings1    537 non-null    object
 7   meanings2    100 non-null    object
 8   common_part  209 non-null    object
dtypes: int64(1), object(8)
memory usage: 429.5+ KB


/tmp/ipykernel_7039/3246353912.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defs["common_part"][i] = common
/tmp/ipykernel_7039/3246353912.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defs["meanings1"][i] = non
/tmp/ipykernel_7039/3246353912.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defs["common_part"][i] = common
/tmp/ipykernel_7039/3246353912.py:32: SettingWithCopyWarning: 
A value is trying to be set on a 

In [5]:
s = set(words2) - set(words1)
s, len(s)

({'а-форфэ',
  'авизо',
  'агрегирование',
  'администрация',
  'ажур',
  'аккредитив',
  'анкета',
  'ассигмент',
  'боны',
  'валютный трансферт',
  'взаимные расчеты',
  'виза',
  'вторичный рынок',
  'гарантированные ценные бумаги',
  'денежная система',
  'дефляция',
  'долевая собственность',
  'домицилирование',
  'допуск',
  'закон больших чисел',
  'закрытая подписка',
  'земельная рента'},
 22)

In [6]:
defs["counts"] = defs["word"].apply(lambda x: len(x.split()))
defs.groupby("counts")["word"].count()

counts
1     1630
2     3209
3      804
4      273
5      107
6       48
7       17
8       11
9        5
10       3
Name: word, dtype: int64

In [7]:
defs_ones = defs.where(defs.counts == 1).dropna(axis = 0, how = 'all')
defs_ones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1630 entries, 0 to 6106
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   1630 non-null   float64
 1   letter       1630 non-null   object 
 2   url          1630 non-null   object 
 3   word         1630 non-null   object 
 4   text1        1630 non-null   object 
 5   text2        377 non-null    object 
 6   meanings1    297 non-null    object 
 7   meanings2    67 non-null     object 
 8   common_part  167 non-null    object 
 9   counts       1630 non-null   float64
dtypes: float64(2), object(8)
memory usage: 140.1+ KB


# Scraping wiktionary

Main point to create dataset for finantial polysemy words with examples

In [16]:
url = "https://ru.wiktionary.org/wiki/"

In [31]:
driver = webdriver.Chrome()

In [135]:
num = 0
cols = ["word", "meaning", "examples", "classes"]
wik = pd.DataFrame(columns=cols)

for i in defs_ones.values:
    
    w = i[3]
    driver.get(url + w)
    try:
        objs = driver.find_element(By.TAG_NAME, "ol").find_elements(By.TAG_NAME, "li")
    except:
        print(f"Not found word {w}")
        continue
    lst = []
    
    num_classes = len(objs)
    for obj in objs:
        
        
        sense = obj.text.split("◆")
        if sense[0] != "":
            examples = sense[1:]
            if len(examples):
                if examples[0] == " Отсутствует пример употребления (см. рекомендации).":
                    examples = None
            else:
                examples = None
            lst.append((w, 
                       sense[0],
                       examples,
                       num_classes))
    
    to_add = pd.DataFrame(lst, columns=cols)
    wik = pd.concat([to_add, wik], ignore_index=True)
    num += 1
    
    if num % 50 == 0:
        print(f"Proccessed {num} words")
    sleep(1)

Not found word авераж
Not found word абекор
Not found word авиаипотека
Not found word аджастер
Proccessed 50 words
Not found word аграрии
Not found word антидатирование
Not found word апорты
Not found word акции-двойники
Not found word андерлаинг
Not found word андинование
Not found word аманко
Not found word актуариат
Not found word а-форфэ
Not found word аргентарии
Not found word ассигмент
Proccessed 100 words
Not found word аленди
Not found word амбаллаж
Not found word аннуитент
Not found word арриарс
Not found word арбитражирование
Not found word аутстаффинг
Not found word аргентарий
Not found word аргументарий
Not found word бай-бэк
Not found word байндер
Not found word бабл
Proccessed 150 words
Not found word банк-учредитель
Not found word баланс-нетто
Not found word бенилюкс
Not found word бесса
Not found word банк-акцептант
Not found word банкиры
Not found word банк-консорциум
Not found word банко
Not found word бибор
Not found word билатеризм
Not found word бланк-ваучер
Not fo

In [136]:
wik

,word,meaning,examples,classes
0,ящик,"полое вместилище в форме параллелепипеда, сдел...",None,7
1,ящик,"ящик [1], являющийся частью предмета мебели, н...",None,7
2,ящик,"количество каких-либо однородных предметов, по...",None,7
3,ящик,"перен., разг., пренебр. то же, что телевизор [...",None,7
4,ящик,"перен., жарг. то же, что предприятие, как прав...",None,7
...,...,...,...,...
2890,авантюра,устар. легкомысленная затея; опасное или необы...,"[ Каульбарсу поступок Попова очень понравился,...",3
2891,авантаж,устар. выгода; преимущество,[ Марья Николаевна в тот день принарядилась оч...,2
2892,аванс,"фин. денежные или иные материальные ценности, ...","[ В пятницу нам выдали аванс по 3000 рублей, а...",3
2893,аванс,"перен. доверие, похвала, поддержка, выраженные...",[ Ему щедро раздавали авансы.],3


In [139]:
wik.to_csv("wik_one_defs.csv", index=False)

In [8]:
defs_nones = defs.where(defs.counts != 1).dropna(axis = 0, how = 'all')
defs_nones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4477 entries, 1 to 6105
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   4477 non-null   float64
 1   letter       4477 non-null   object 
 2   url          4477 non-null   object 
 3   word         4477 non-null   object 
 4   text1        4476 non-null   object 
 5   text2        718 non-null    object 
 6   meanings1    240 non-null    object 
 7   meanings2    33 non-null     object 
 8   common_part  42 non-null     object 
 9   counts       4477 non-null   float64
dtypes: float64(2), object(8)
memory usage: 384.7+ KB


In [32]:
num = 0
cols = ["word", "meaning", "examples", "classes"]
wik_nones = pd.DataFrame(columns=cols)
for i in defs_nones.values:
    
    word = i[3]
    ws = word.split() + [word.replace(" ", "_")]
    lst = []
    
    for w in ws:
        driver.get(url + w)
        try:
            objs = driver.find_element(By.TAG_NAME, "ol").find_elements(By.TAG_NAME, "li")
        except:
            print(f"Not found word {w}")
            continue
        
        num_classes = len(objs)
        for obj in objs:
            
            sense = obj.text.split("◆")
            if sense[0] != "":
                examples = sense[1:]
                if len(examples):
                    if examples[0] == " Отсутствует пример употребления (см. рекомендации).":
                        examples = None
                else:
                    examples = None
                
                lst.append((w, 
                           sense[0],
                           examples,
                           num_classes))
        
    to_add = pd.DataFrame(lst, columns=cols)
    wik_nones = pd.concat([to_add, wik_nones], ignore_index=True)
    num += 1
    
    if num % 450 == 0:
        print(f"Proccessed {num} words")
    sleep(1)

Not found word рынок_монополистической_конкуренции
Not found word обочине
Not found word рынок_на_обочине
Not found word (англ,
Not found word rating)
Not found word рейтинг_(англ,_rating)
Not found word рейтинг_кредитоспособности_банка
Not found word халберта
Not found word рейтинг_халберта
Not found word рейтинг_ценных_бумаг
Not found word ретрективные
Not found word ретрективные_ценные_бумаги
Not found word (от
Not found word фр.
Not found word retrocession
Not found word переуступка)
Not found word ретроцессия_(от_фр._retrocession_—_переуступка)
Not found word (лат.
Not found word referens
Not found word сообщающий)
Not found word референт_(лат._referens_—_сообщающий)
Not found word товара
Not found word рынок_наличного_товара
Not found word покупателя
Not found word рынок_покупателя
Not found word принципалов
Not found word рынок_принципалов
Not found word продавца
Not found word рынок_продавца
Not found word производителя
Not found word рынок_производителя
Not found word рынок_сп

In [34]:
driver.quit()

In [35]:
wik_nones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32266 entries, 0 to 32265
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   word      32266 non-null  object
 1   meaning   32266 non-null  object
 2   examples  23151 non-null  object
 3   classes   32266 non-null  object
dtypes: object(4)
memory usage: 1008.4+ KB


In [41]:
wik_nones.to_csv("wik_nones_defs.csv", index=False)